In [2]:
from glob import glob
import pandas as pd

# first import all GS files and create a nice datastructure
df = pd.DataFrame(columns=['id','category','values'])
for name in glob('/Users/marieke/Work/Food/Jamie_LOD_iver/data/annotations/ba*/*csv'):
    name_elems = name.split("/")
    file2 = pd.read_csv(name)
    for hip, hop in file2.iterrows():
        if hop['TYPE'] == "PERSON":
            row2 = [name_elems[9][:-4], 'INGREDIENTS', name_elems[9][:-4]+"_"+hop["QUOTE_TRANSCRIPTION"].lower() ] 
            df.loc[len(df) ] = row2
            row2 = []
        if hop['TYPE'] == "PLACE":
            row3 = [name_elems[9][:-4],  'QUANTITIES', name_elems[9][:-4]+"_"+hop["QUOTE_TRANSCRIPTION"].lower() ] 
            df.loc[len(df) ] = row3
            row3 = []
        if hop['TYPE'] == "EVENT":
            row4 = [name_elems[9][:-4],  'OCR', name_elems[9][:-4]+"_"+hop["QUOTE_TRANSCRIPTION"].lower() ] 
            df.loc[len(df)  ] = row4
            row4 = [] 

In [3]:
df.head()

,id,category,values
0,1389,INGREDIENTS,1389_munt
1,1389,INGREDIENTS,1389_pruimen
2,1389,INGREDIENTS,1389_okra's
3,1389,INGREDIENTS,1389_tomaten
4,1389,QUANTITIES,1389_3 eetlepels


In [4]:
huppeldepupje = pd.DataFrame(columns=['id','category', 'ingr_set'])
huppeldepupje = df.groupby(['id','category'])['values'].apply(set)

In [5]:
huppeldepupje.head()

id    category   
1042  INGREDIENTS    {1042_zout, 1042_sherry, 1042_thijm, 1042_bief...
      OCR            {1042_ken-ie, 1042_pakrje, 1042_aardbeienoharl...
      QUANTITIES     {1042_100 gram, 1042_1 kleine, 1042_enkele, 10...
1119  INGREDIENTS    {1119_brandneteltopjes, 1119_kip, 1119_spinazi...
1127  INGREDIENTS    {1127_wortel, 1127_ui, 1127_selderiestengels, ...
Name: values, dtype: object

In [6]:
newframe = huppeldepupje.to_frame()
newnewframe = newframe.reset_index(level='category')
newframe = newnewframe.reset_index(level='id')
newframe.head()

,id,category,values
0,1042,INGREDIENTS,"{1042_zout, 1042_sherry, 1042_thijm, 1042_bief..."
1,1042,OCR,"{1042_ken-ie, 1042_pakrje, 1042_aardbeienoharl..."
2,1042,QUANTITIES,"{1042_100 gram, 1042_1 kleine, 1042_enkele, 10..."
3,1119,INGREDIENTS,"{1119_brandneteltopjes, 1119_kip, 1119_spinazi..."
4,1127,INGREDIENTS,"{1127_wortel, 1127_ui, 1127_selderiestengels, ..."


In [7]:
def import_list():
## IMPORT NEW LIST
# import the list of ingredients
# Complete list seems to result in a slightly higher recall
	list_ = []
#	with open("ingredient_list.csv", 'r') as x: # yes it's hard coded, not so neat
	with open("/data/ingredients_lists/unique_ingredients_ah.csv", 'r') as x: # yes it's hard coded, not so neat
		for line in x:
			line = line.rstrip()
			elems = line.split(",")
			list_.append(" ".join(elems[:-1]))
#	print(list_)
	return(list_)

def preprocess_text(t):
	tagged_list_ = []
	tokenizer = RegexpTokenizer(r'\w+')
	words = tokenizer.tokenize(t)
	words = " ".join(words)
	return(words)

def detect_ingredients(text,list_):
	ingredients = []
	for x in list_:
		x = " " + x +" "
		ingredient = re.search(x, text)
		if ingredient:
			cleanright = ingredient.group().rstrip()
			cleanleft = cleanright.lstrip()
			lowercase = cleanleft.lower()            
			ingredients.append(lowercase)
		unique_ingredients = set(ingredients)
#	print("bla", unique_ingredients)
	return(unique_ingredients)


def detect_quantities(text,ing_list):
# Loop through the text again with the ingredients and see if you can find a quantity
    quantities_ingr = []
    ingredients_matched_copy = []
    metrics = []
    with open("data/ingredients_lists/unique_units.csv", 'r') as x: # yes it's hard coded, not so neat
        for line in x:
            line = line.rstrip()
            metrics.append(line)
#return(metrics)
#metrics = ['dl', 'gr', 'gram', 'eetlepel', 'el', 'theelepel', 'tl', 'snufje', 'snuf', 'ml', 'g']
    for y in ing_list:
        for z in metrics:
            z_plural = pluralize(z)
            quantity = re.search('[0-9]* ' + y + '| [0-9]* ' + z + ' \w*? ' + y + '| [0-9]* ' + z_plural + ' \w*? ' + y + '| twee ' + z_plural + ' \w*? ' + y + 'een* ' + y + '| een ' + z + ' \w*? ' + y, text)
            if quantity:
                quantities_ingr.append(quantity.group())
                ingredients_matched_copy.append(y)
               # print(quantity.group())
# Do something smart so you get one list with items for which you found quantities and items for which you didn't
    copy_ingrs = ing_list
    for item in ingredients_matched_copy:
        try:
            copy_ingrs.remove(item)
        except:
            pass
    merged = list(copy_ingrs) + quantities_ingr
    unique = set(merged)
    return(unique)


def extract_ingredients(t):
	processed_text = preprocess_text(t)
	found_ingredients = detect_ingredients(processed_text, allerhande_ingredients)
	ingr_and_quant = detect_quantities(processed_text, found_ingredients)
	return(ingr_and_quant)

# Read in the ingredients lists 
allerhande_ingredients = import_list()





peterselie 1
koriander 1
basilicum 1
munt 1
bieslook 1
platte peterselie 2
olijfolie 1
rucola 1
crème fraîche 2
slagroom 1
boter 1
dille 1
Parmezaanse kaas 2
knoflook 1
poedersuiker 1
zwarte olijven 2
frambozen 1
Griekse yoghurt 2
spinazie 1
olijfolie extra vierge 3
seroendeng 1
zoete chilisaus 2
suiker 1
kappertjes 1
geitenkaas 1
olie 1
mozzarella 1
melk 1
bosuitjes 1
zonnebloemolie 1
geraspte kaas 2
balsamicoazijn 1
walnoten 1
zachte geitenkaas 2
aardbeien 1
tijm 1
pittige kaas 2
amandelschaafsel 1
water 1
veldsla 1
kropsla 1
pure chocolade 2
waterkers 1
tuinerwten 1
witte kaas 2
citroen 1
chilipeper 1
pistachenoten 1
honing-mosterddressing 1
mayonaise 1
mascarpone 1
selderij 1
citroensap 1
zalmfilet 1
bosuitje 1
roomboter 1
gebakken uitjes 2
gembersiroop 1
oregano 1
tomaten 1
bloem 1
broccoli 1
rozemarijn 1
Japanse sojasaus 2
cherrytomaten 1
pijnboompitten 1
gemalen belegen kaas 3
kippenbouillon 1
Hollandse garnalen 2
cacaopoeder 1
nootmuskaat 1
kraanwater 1
sojasaus 1
cashewnoten 1

yoghurt Griekse stijl 0% vet 5
geroosterd sesamzaad 2
rozenlikeur 1
olijfolie om te bestrijken 4
shoarmavlees 1
volkorenbroodje 1
krulpeterselie 1
eendenhaasje 1
mini marshmellows 2
pastaschelpjes 1
portobello 1
vanillearoma 1
Spaanse tijmhoning 2
gemengde vlokken 2
ijswafeltjes 1
gedroogde chilipeper 2
gedroogde chilipeper uit molentje 4
kipfilets 1
whisky slijter 2
tahini 1
Maestrichter fruitstroop 2
Italiaanse bol 2
rauwkost bietjes 2
pestovinaigrette 1
MonChou verse roomkaas 3
moesappel 1
mix wok sauce oester knoflook 5
geraspte kaas voor ovengratin 4
witbier 1
kokosstukjes 1
geraspte emmentaler kaas 3
filobladerdeeg 1
kokosmakroon 1
tortillawrap 1
korma currypasta 2
blue d'auvergne 50+ 3
sorbetijs citroen 2
geraspte citroenschil 2
cocktailsaus met Schotse wisky 4
bloemkool- en broccoliroosjes 3
tofu roerbakgehakt 2
studentenhaver 1
bieslookpesto 1
gerookte makreelfilet met peper 4
cervelaatworst 1
pastagroente 1
groene peperkorrels 2
Catalaanse worstjes 2
klopfix 1
yoghurtdressing

In [9]:
import re
from nltk.tokenize import sent_tokenize, RegexpTokenizer
from pattern.nl import pluralize, singularize

system_output = {} 
for filename in glob('/data/IngredientQuantitiesUnits_Annotations/EvaluationSet/ba*/*txt'):
    file = open(filename, "r") 
    name_elems = filename.split("/")
    artikel = file.read()

    ing = preprocess_text(artikel)
   # print(ing)
    found = detect_ingredients(ing, allerhande_ingredients)
    #qua = detect_quantities(ing, found)
    found_list = list(found)
    inbetween = []
    for x in found_list:
        inbetween.append(name_elems[9][:-4] + "_" + x)
    found = set(inbetween)
    #print(found)
    #ids = []
    #for y in qua:
    #    ids.append(name_elems[9][:-4] + "_" + y)
    #quants_found = set(ids)
    found = set(inbetween)
    inbetween = []
    #ids = []
    system_output[name_elems[9][:-4]] = found
    #print(system_output[name_elems[9][:-4]])
    #system_output[name_elems[9][:-4]] = quants_found
    found = {}
    #quants_found = {}
    

In [10]:
def prec_r_f(a1,a2):
    tp = a1.intersection(a2)
    fp = a2.difference(a1)
    fn = a1.difference(a2)
    precision = len(tp) / (len(tp) + len(fp))
    recall = len(tp) / (len(tp) + len(fn)) 
    f = (precision + recall) / 2 
    answer = "\tItemsGS:\t" + str(len(a1)) + "\tItemsSystem:\t" + str(len(a2)) + "\tcorrect:\t" + str(len(tp))+ "\tp:\t" + str(precision) + "\tr:\t" + str(recall) + "\tf1:\t" + str(f) + "\nin SystemOutput not in GS" + str(fp) + "\nin GS not in SystemOutput" + str(fn) 
    return(answer)

{'1042': {'1042_aardbeien',
  '1042_biefstuk',
  '1042_brood',
  '1042_champignons',
  '1042_eidooiers',
  '1042_eieren',
  '1042_geraspte',
  '1042_gin',
  '1042_lange vingers',
  '1042_mosterd',
  '1042_olie',
  '1042_paneermeel',
  '1042_paprika',
  '1042_peper',
  '1042_rijst',
  '1042_rode paprika',
  '1042_room',
  '1042_saus',
  '1042_sherry',
  '1042_sla',
  '1042_slagroom',
  '1042_suiker',
  '1042_tijm',
  '1042_tomaat',
  '1042_tomaten',
  '1042_ui',
  '1042_uien',
  '1042_vanillepudding',
  '1042_vanillesuiker',
  '1042_witte',
  '1042_zout',
  '1042_zout en peper'},
 '1119': {'1119_kip', '1119_olie', '1119_spinazie', '1119_water'},
 '1127': {'1127_aardappelen',
  '1127_bloem',
  '1127_cognac',
  '1127_garnalen',
  '1127_olie',
  '1127_peper',
  '1127_peterselie',
  '1127_rijst',
  '1127_room',
  '1127_tomaten',
  '1127_ui',
  '1127_uien',
  '1127_water',
  '1127_witte',
  '1127_witte wijn',
  '1127_wortel',
  '1127_zout'},
 '1150': {'1150_azijn',
  '1150_boter',
  '1150_br

In [13]:
sysout_aggregated = []
for x in system_output:
    for y in system_output[x]:
        sysout_aggregated.append(y)
        
sysout_agg = set(sysout_aggregated)


In [14]:
for x in system_output:
    gold_standard = newframe.loc[(newframe['category'] == 'INGREDIENTS') & (newframe['id'] == x) ]['values'].tolist()
    try:
        lijstje = set(gold_standard[0])
        prf = prec_r_f(lijstje,system_output[x])
        print(prf)
    except:
     #   lijstje = set(['0'])
     #   prf = prec_r_f(lijstje,system_output[x])
     #   print(prf)
        print("Something's wrong here: ", x)

	ItemsGS:	18	ItemsSystem:	17	correct:	15	p:	0.8823529411764706	r:	0.8333333333333334	f1:	0.857843137254902
in SystemOutput not in GS{'1127_water', '1127_witte'}
in GS not in SystemOutput{'1127_selderiestengels', '1127_winterwortel', '1127_bleekselderiestengels'}
	ItemsGS:	17	ItemsSystem:	7	correct:	6	p:	0.8571428571428571	r:	0.35294117647058826	f1:	0.6050420168067226
in SystemOutput not in GS{'1293_cake'}
in GS not in SystemOutput{'1293_delfia', '1293_plantenmargarine', '1293_delfrite', '1293_braadvet', '1293_kaakjes', '1293_aardnoten', '1293_biscuits', '1293_mariakaakjes', '1293_koffie-extract', '1293_biscuit', '1293_delfiatablet'}
	ItemsGS:	5	ItemsSystem:	4	correct:	2	p:	0.5	r:	0.4	f1:	0.45
in SystemOutput not in GS{'1119_olie', '1119_water'}
in GS not in SystemOutput{'1119_brandnetels', '1119_brandneteltopjes', '1119_molsla'}
	ItemsGS:	45	ItemsSystem:	32	correct:	26	p:	0.8125	r:	0.5777777777777777	f1:	0.6951388888888889
in SystemOutput not in GS{'1042_geraspte', '1042_rode paprika',

In [15]:
# Also get aggregated scores to put in the paper, the above is handy for error analysis
aggregated = pd.DataFrame(columns=['category', 'ingr_set'])
aggregated = df.groupby(['category'])['values'].apply(set)
aggframe = aggregated.to_frame()
aggframe_reset = aggframe.reset_index(level='category')
aggframe_reset

,category,values
0,INGREDIENTS,"{3840_knoflookteentje, 1465_bloem, 1293_delfri..."
1,OCR,"{425_ll* cc, 7234_msn ippers, 6842_kiiof«eentj..."
2,QUANTITIES,"{6380_zestal, 4978_5, 3553_potje, 1042_achtste..."


In [31]:
gs_agg_ingr = aggframe_reset.loc[(aggframe_reset['category'] == 'INGREDIENTS')  ]['values'].tolist()
gs_agg_ocr = aggframe_reset.loc[(aggframe_reset['category'] == 'OCR')  ]['values'].tolist()
gs_agg_ocr_ing = (gs_agg_ocr[0] | gs_agg_ingr[0])
#agg_list_ingr = set(gs_agg_ingr[0])
prf_agg_ingr = prec_r_f(gs_agg_ocr_ing,sysout_agg)
print(prf_agg_ingr)

	ItemsGS:	1686	ItemsSystem:	1426	correct:	998	p:	0.699859747545582	r:	0.5919335705812574	f1:	0.6458966590634196
in SystemOutput not in GS{'4411_koud water', '2751_zout en peper', '4612_amsterdamse', '1716_chinese', '6401_rijst', '5568_kokend water', '4391_honing', '6767_rijst', '3686_ananas', '5568_zout en peper', '6428_chocoladecake', '6380_selderij', '4997_heet water', '6541_witte', '6849_zout en versgemalen zwarte peper', '6767_vissaus', '5259_boter om in te vetten', '4612_kaas', '4612_peper', '1652_kaas', '1389_olie of boter', '2192_chinese', '1176_zoete', '874_water', '324_boter', '4278_geraspte', '995_dipsaus', '2857_versgemalen peper', '4278_room', '2089_saus', '3370_water', '1343_hollandaisesaus', '6234_koud water', '3678_koekjes', '5673_knoflook', '6767_chinese', '2226_zout en peper', '2686_geraspte oude kaas', '5893_peper', '1815_witte', '4710_rijst', '995_pittige mosterd', '6849_versgemalen zwarte peper', '5673_franse', '7235_zoete', '4467_water', '3823_tomaat', '1563_brood'

/Users/marieke/Work/Food/Jamie_LOD_iver/data/annotations/batch1_marieke/995.txt
knoflook
suiker
olie
walnoten
pijnboompitten
sojasaus
sesamzaad
sesamolie
kipfilet
mosterd
kaneel
azijn
zout
hazelnoten
amandelen
gemalen kaneel
sesamzaadjes
gember
eieren
peper
uien
zwarte peper
rijst
rijstazijn
lichte sojasaus
pittige mosterd
runderbouillon
biefstuk
Chinese
gemalen gember
worteltjes
komkommersalade
kabeljauw
walnoten
dipsaus
/Users/marieke/Work/Food/Jamie_LOD_iver/data/annotations/batch1_marieke/798.txt
slagroom
suiker
aardbeien
water
yoghurt
Ananas
koud water
gelatine
eidooiers
kwark
kiwi s
Kiwi
eiwitten
/Users/marieke/Work/Food/Jamie_LOD_iver/data/annotations/batch1_marieke/372.txt
peterselie
boter
suiker
water
mayonaise
citroensap
bloem
paprikapoeder
mosterd
azijn
zout
peper
bananen
ui
aardappelen
rijst
banaan
kruiden
margarine
boter of margarine
saus
kerrie
worteltjes
fijngehakte peterselie
gehaktballetjes
/Users/marieke/Work/Food/Jamie_LOD_iver/data/annotations/batch1_marieke/945.txt

{'995': {'995_dipsaus', '995_sesamzaadjes', '995_gemalen kaneel', '995_runderbouillon', '995_Chinese', '995_zout', '995_sojasaus', '995_knoflook', '995_sesamolie', '995_sesamzaad', '995_pittige mosterd', '995_suiker', '995_azijn', '995_worteltjes', '995_rijst', '995_pijnboompitten', '995_kaneel', '995_biefstuk', '995_rijstazijn', '995_zwarte peper', '995_hazelnoten', '995_eieren', '995_komkommersalade', '995_olie', '995_kabeljauw', '995_mosterd', '995_peper', '995_kipfilet', '995_gember', '995_uien', '995_lichte sojasaus', '995_walnoten', '995_amandelen', '995_gemalen gember'}, '798': {'798_eidooiers', '798_gelatine', '798_suiker', '798_eiwitten', '798_water', '798_kiwi s', '798_koud water', '798_Kiwi', '798_aardbeien', '798_yoghurt', '798_kwark', '798_slagroom', '798_Ananas'}, '372': {'372_citroensap', '372_kerrie', '372_water', '372_fijngehakte peterselie', '372_ui', '372_peterselie', '372_saus', '372_bananen', '372_margarine', '372_suiker', '372_rijst', '372_gehaktballetjes', '372_b

	ItemsGS:	1536	ItemsSystem:	187	correct:	131	p:	0.7005347593582888	r:	0.08528645833333333	f1:	0.3929106088458111
in SystemOutput not in GS{'995_dipsaus', '320_geraspte', '945_chocolademousse', '995_gemalen kaneel', '874_kaas', '995_Chinese', '945_kokend water', '874_boter of margarine', '798_Kiwi', '874_geraspte', '995_sojasaus', '995_gemalen gember', '247_Italiaanse', '372_fijngehakte peterselie', '320_Italiaanse', '320_gepelde tomaten', '324_witte', '320_witte', '995_pittige mosterd', '945_water', '320_takje tijm', '798_kiwi s', '425_versgemalen peper', '372_saus', '874_kokend water', '320_gezeefde tomaten', '874_saus', '874_roomsaus', '372_kruiden', '320_zout en peper', '874_Italiaanse', '874_geraspte gruyère', '874_peper en zout', '324_boter', '798_koud water', '247_zachte', '874_water', '802_pinda s', '324_water', '372_boter of margarine', '995_komkommersalade', '995_olie', '425_kokend water', '425_appel', '320_fijngehakte peterselie', '320_citroen', '425_witte', '324_bier', '945_

In [32]:
from pattern.nl import pluralize, singularize

def import_list():
## IMPORT NEW LIST
# import the list of ingredients
# Complete list seems to result in a slightly higher recall
    dbplist = []
#	with open("ingredient_list.csv", 'r') as x: # yes it's hard coded, not so neat
    with open("/data/ingredients_lists/dbpedia_ingredients.tsv", 'r') as x: # yes it's hard coded, not so neat
        for line in x:
            line = line.rstrip()
            elems = line.split("\t")
            no_parentheses = elems[1].split("(")
            try_singular = singularize(no_parentheses[0])
            if no_parentheses[0].lower() == try_singular and pluralize(no_parentheses[0])[-3:] != " en":
                dbplist.append(no_parentheses[0].lower())
                dbplist.append(pluralize(no_parentheses[0]))
            else:
                dbplist.append(no_parentheses[0].lower())
            if len(elems[1]) < 1:
                elems[0] = elems[0].replace('http://nl.dbpedia.org/resource/', '')
                elems[0] = elems[0].replace("_", " ")
                no_parentheses = elems[0].split("(")
                try_singular = singularize(no_parentheses[0])
                if no_parentheses[0].lower() == try_singular:
                    if pluralize(no_parentheses[0])[-3:] != " en":
                        dbplist.append(no_parentheses[0].lower())
                        dbplist.append(pluralize(no_parentheses[0])) 
                else:
                    dbplist.append(no_parentheses[0].lower())
    templist = set(dbplist)
    try:
        templist.remove('en')
    except:
        pass
    dbplist = list(templist)
    return(dbplist)

dbpl = import_list()


bla_ en
4110


In [34]:
def preprocess_text(t):
	tagged_list_ = []
	tokenizer = RegexpTokenizer(r'\w+')
	words = tokenizer.tokenize(t)
	words = " ".join(words)
	return(words)

def detect_ingredients(text,lijst):
	ingredients = []
	for x in lijst:
		x = " " + x +" "
		ingredient = re.search(x, text)
		if ingredient:
			cleanright = ingredient.group().rstrip()
			cleanleft = cleanright.lstrip()
			lowercase = cleanleft.lower()            
			ingredients.append(lowercase)
		unique_ingredients = set(ingredients)
#	print("bla", unique_ingredients)
	return(unique_ingredients)


def detect_quantities(text,ing_list):
# Loop through the text again with the ingredients and see if you can find a quantity
	quantities_ingr = []
	ingredients_matched_copy = []
	metrics = []
	with open("/data/ingredients_lists/unique_units.csv", 'r') as x: # yes it's hard coded, not so neat
		for line in x:
			line = line.rstrip()
			metrics.append(line)
	#return(metrics)
	#metrics = ['dl', 'gr', 'gram', 'eetlepel', 'el', 'theelepel', 'tl', 'snufje', 'snuf', 'ml', 'g']
	for y in ing_list:
		for z in metrics:
			my_regex = "[0-9]* " + z + y + "|[0-9]* " + z + y + "|[0-9]+" + y
			quantity = re.search(my_regex, text)
			if quantity:
				quantities_ingr.append(quantity.group())
				ingredients_matched_copy.append(y)
	# Do something smart so you get one list with items for which you found quantities and items for which you didn't
	copy_ingrs = ing_list
	for item in ingredients_matched_copy:
		try:
			copy_ingrs.remove(item)
		except:
			pass
	merged = list(copy_ingrs) + quantities_ingr
	unique = set(merged)
	return(unique)

def extract_ingredients(t):
	processed_text = preprocess_text(t)
	found_ingredients = detect_ingredients(processed_text, dbpl)
	ingr_and_quant = detect_quantities(processed_text, found_ingredients)
	return(ingr_and_quant)

In [35]:
import re
from nltk.tokenize import sent_tokenize, RegexpTokenizer
from glob import glob

system_output = {} 
for filename in glob('/data/IngredientQuantitiesUnits_Annotations/EvaluationSet/ba*/*txt'):
    file = open(filename, "r") 
    name_elems = filename.split("/")
    artikel = file.read()

    ing = preprocess_text(artikel)
    #print(ing)
    found = detect_ingredients(ing, dbpl)
#	qua = detect_quantities(ing, found)
    found_list = list(found)
    inbetween = []
    for x in found_list:
        inbetween.append(name_elems[9][:-4] + "_" + x)
    found = set(inbetween)
    inbetween = []
    system_output[name_elems[9][:-4]] = found
    found = {}

In [36]:
aggregated = pd.DataFrame(columns=['category', 'ingr_set'])
aggregated = df.groupby(['category'])['values'].apply(set)
aggframe = aggregated.to_frame()
aggframe_reset = aggframe.reset_index(level='category')
aggframe_reset

,category,values
0,INGREDIENTS,"{3840_knoflookteentje, 1465_bloem, 1293_delfri..."
1,OCR,"{425_ll* cc, 7234_msn ippers, 6842_kiiof«eentj..."
2,QUANTITIES,"{6380_zestal, 4978_5, 3553_potje, 1042_achtste..."


In [38]:
sysout_aggregated = []
for x in system_output:
    for y in system_output[x]:
        sysout_aggregated.append(y)
        
sysout_agg = set(sysout_aggregated)

In [39]:
sysout_aggregated = []
for x in system_output:
    for y in system_output[x]:
        sysout_aggregated.append(y)
        
sysout_agg = set(sysout_aggregated)

In [40]:
#gs_agg_ingr = aggframe_reset.loc[((aggframe_reset['category'] == 'INGREDIENTS') | (aggframe_reset['category'] == 'OCR')) ]['values'].tolist()
#agg_list_ingr = set(gs_agg_ingr[0])
#prf_agg_ingr = prec_r_f(agg_list_ingr,sysout_agg)
#print(prf_agg_ingr)


gs_agg_ingr = aggframe_reset.loc[(aggframe_reset['category'] == 'INGREDIENTS')  ]['values'].tolist()
gs_agg_ocr = aggframe_reset.loc[(aggframe_reset['category'] == 'OCR')  ]['values'].tolist()
gs_agg_ocr_ing = (gs_agg_ocr[0] | gs_agg_ingr[0])
#agg_list_ingr = set(gs_agg_ingr[0])
prf_agg_ingr = prec_r_f(gs_agg_ocr_ing,sysout_agg)
print(prf_agg_ingr)

	ItemsGS:	1686	ItemsSystem:	855	correct:	513	p:	0.6	r:	0.304270462633452	f1:	0.45213523131672595
in SystemOutput not in GS{'1363_lever', '1293_taart', '4391_honing', '6694_vet', '4467_soepen', '3686_ananas', '3678_chinese bieslook', '324_drank', '1652_vet', '1389_zon', '6380_selderij', '247_snack', '6767_vissaus', '324_soep', '4612_kaas', '4612_peper', '4467_groente', '1652_kaas', '6541_groenten', '798_zon', '874_water', '1389_soep', '1293_stollen', '3475_zon', '995_dipsaus', '3686_seitan', '2089_blikje', '3840_veldzuring', '2089_saus', '1716_drank', '3370_water', '3925_vet', '3686_vleesvervangers', '5673_knoflook', '2857_ingrediënten', '5893_peper', '3395_toastjes', '1119_soep', '4799_laurier', '7034_champignon', '7419_wijn', '320_wijn', '4278_wijn', '4467_water', '995_vlees', '1901_dier', '3823_tomaat', '1563_brood', '6401_vlees', '1363_fruit', '5893_water', '247_ingrediënt', '3686_groente', '5366_brood', '6428_nagerecht', '1363_groente', '798_zuivel', '5549_zon', '4622_hapje', '1127